In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
df=pd.read_excel('MOOD TRAK.xlsx')
df.head()

,Time Stamp,Day,Activities,Start,End,Quality,Note
0,2024-05-01 07:00:00,Wednesday,sleep,23:00:00,05:00:00,okay,Bangun pagi untuk sholat subuh.
1,2024-05-01 07:00:00,Wednesday,selfcare,05:00:00,05:30:00,good,"Mandi dan bersiap, air dingin di pagi hari tid..."
2,2024-05-01 07:00:00,Wednesday,eating,05:30:00,06:00:00,good,Sarapan pagi dengan roti bakar dan susu.
3,2024-05-01 07:00:00,Wednesday,workout,06:00:00,07:00:00,excellent,Jogging sekitar kos-kosan. Bertemu banyak anak...
4,2024-05-01 07:00:00,Wednesday,traveling,07:00:00,07:30:00,excellent,"Berangkat ke kampus, perjalanan sekitar 2 km d..."


In [ ]:
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model('day_condition_model.h5')

In [ ]:
df_new=df.drop(columns=['Note'])
# Convert time columns to datetime
df_new['Time Stamp'] = pd.to_datetime(df_new['Time Stamp']).dt.date
# Map quality ratings to numeric values
quality_mapping = {'terrible': 0, 'bad': 1, 'okay': 2, 'good': 3, 'excellent': 4}
df_new['Quality'] = df_new['Quality'].map(quality_mapping)

# Convert 'Start' and 'End' to datetime
df_new['Start'] = pd.to_datetime(df_new['Start'], format='%H:%M:%S').dt.time
df_new['End'] = pd.to_datetime(df_new['End'], format='%H:%M:%S').dt.time

# Calculate duration
df_new['Duration'] = pd.to_timedelta(df_new['End'].astype(str)) - pd.to_timedelta(df_new['Start'].astype(str))
df_new['Duration'] = df_new['Duration'].apply(lambda x: x + pd.Timedelta(days=1) if x.days < 0 else x)

df_new['Duration'] = df_new['Duration'].apply(lambda x: int(pd.to_timedelta(x).total_seconds() // 60))

df_new

,Time Stamp,Day,Activities,Start,End,Quality,Duration
0,2024-05-01,Wednesday,sleep,23:00:00,05:00:00,2,360
1,2024-05-01,Wednesday,selfcare,05:00:00,05:30:00,3,30
2,2024-05-01,Wednesday,eating,05:30:00,06:00:00,3,30
3,2024-05-01,Wednesday,workout,06:00:00,07:00:00,4,60
4,2024-05-01,Wednesday,traveling,07:00:00,07:30:00,4,30
...,...,...,...,...,...,...,...
1215,2024-08-06,Tuesday,entertainment,17:45:00,19:00:00,2,75
1216,2024-08-06,Tuesday,eating,19:00:00,19:30:00,2,30
1217,2024-08-06,Tuesday,study,19:30:00,21:00:00,2,90
1218,2024-08-06,Tuesday,dating,21:00:00,23:00:00,2,120


In [ ]:
# Aggregate daily data
daily_summary = df_new.groupby(['Time Stamp', 'Activities']).agg({'Duration': 'sum'}).reset_index()

daily_summary_pivot = daily_summary.pivot(index='Time Stamp', columns='Activities', values='Duration').fillna(0)
daily_summary_pivot.reset_index(inplace=True)

daily_summary_pivot

Activities,Time Stamp,dating,eating,entertainment,selfcare,sleep,study,traveling,work,workout
0,2024-05-01,0.0,105.0,165.0,45.0,360.0,630.0,135.0,0.0,60.0
1,2024-05-02,260.0,135.0,0.0,45.0,300.0,480.0,145.0,0.0,75.0
2,2024-05-03,0.0,120.0,180.0,60.0,480.0,480.0,60.0,0.0,60.0
3,2024-05-04,210.0,90.0,300.0,45.0,795.0,0.0,0.0,0.0,0.0
4,2024-05-05,150.0,85.0,465.0,0.0,300.0,240.0,200.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
93,2024-08-02,120.0,120.0,75.0,30.0,450.0,90.0,75.0,465.0,45.0
94,2024-08-03,120.0,120.0,75.0,30.0,450.0,90.0,75.0,435.0,45.0
95,2024-08-04,120.0,120.0,75.0,30.0,420.0,90.0,75.0,435.0,45.0
96,2024-08-05,120.0,120.0,75.0,30.0,420.0,90.0,75.0,465.0,45.0


In [ ]:
X_new = daily_summary_pivot.drop(columns=['Time Stamp'])

In [ ]:
# Make predictions
predictions = model.predict(X_new)

# Add the predicted classes to the new dataset
daily_summary_pivot['Predicted_Day_Condition'] = predictions.argmax(axis=1)

# Create a new LabelEncoder instance
label_encoder = LabelEncoder()
original_class_labels = ['Active Explorer', 'Balanced Achiever', 'Fitness Fanatic', 'Stressful Overload']
# Fit the LabelEncoder to the original class labels
label_encoder.fit(original_class_labels)

# Inverse transform the predicted classes to get the original class labels
daily_summary_pivot['Predicted_Day_Condition'] = label_encoder.inverse_transform(daily_summary_pivot['Predicted_Day_Condition'])

# Print or save the new dataset with predicted class labels
daily_summary_pivot

4/4 [==============================] - 0s 8ms/step


Activities,Time Stamp,dating,eating,entertainment,selfcare,sleep,study,traveling,work,workout,Predicted_Day_Condition
0,2024-05-01,0.0,105.0,165.0,45.0,360.0,630.0,135.0,0.0,60.0,Stressful Overload
1,2024-05-02,260.0,135.0,0.0,45.0,300.0,480.0,145.0,0.0,75.0,Fitness Fanatic
2,2024-05-03,0.0,120.0,180.0,60.0,480.0,480.0,60.0,0.0,60.0,Balanced Achiever
3,2024-05-04,210.0,90.0,300.0,45.0,795.0,0.0,0.0,0.0,0.0,Balanced Achiever
4,2024-05-05,150.0,85.0,465.0,0.0,300.0,240.0,200.0,0.0,0.0,Active Explorer
...,...,...,...,...,...,...,...,...,...,...,...
93,2024-08-02,120.0,120.0,75.0,30.0,450.0,90.0,75.0,465.0,45.0,Active Explorer
94,2024-08-03,120.0,120.0,75.0,30.0,450.0,90.0,75.0,435.0,45.0,Active Explorer
95,2024-08-04,120.0,120.0,75.0,30.0,420.0,90.0,75.0,435.0,45.0,Active Explorer
96,2024-08-05,120.0,120.0,75.0,30.0,420.0,90.0,75.0,465.0,45.0,Active Explorer


In [ ]:
# Define thresholds
sleep_threshold = 7 * 60
study_threshold = 8 * 60
work_threshold = 8 * 60
selfcare_threshold = 1 * 60
traveling_threshold = 1 * 60
workout_threshold = 1 * 60
entertainment_threshold = 1 * 60
eating_threshold = 1 * 60

def classify_day(row):
    sleep = row.get('sleep', 0)
    study = row.get('study', 0)
    dating = row.get('dating', 0)
    work = row.get('work', 0)
    selfcare = row.get('selfcare', 0)
    traveling = row.get('traveling', 0)
    workout = row.get('workout', 0)
    entertainment = row.get('entertainment', 0)
    eating = row.get('eating', 0)

    if sleep < sleep_threshold and work > work_threshold:
        return 'Stressful Overload'
    elif sleep < sleep_threshold and study > study_threshold:
        return 'Stressful Overload'
    elif workout > workout_threshold and eating > eating_threshold:
        return 'Fitness Fanatic'
    elif traveling > traveling_threshold and entertainment > entertainment_threshold:
        return 'Active Explorer'
    else:
        return 'Balanced Achiever'
# Apply classification
daily_summary_pivot['Day_Condition'] = daily_summary_pivot.apply(classify_day, axis=1)
daily_summary_pivot

Activities,Time Stamp,dating,eating,entertainment,selfcare,sleep,study,traveling,work,workout,Predicted_Day_Condition,Day_Condition
0,2024-05-01,0.0,105.0,165.0,45.0,360.0,630.0,135.0,0.0,60.0,Stressful Overload,Stressful Overload
1,2024-05-02,260.0,135.0,0.0,45.0,300.0,480.0,145.0,0.0,75.0,Fitness Fanatic,Fitness Fanatic
2,2024-05-03,0.0,120.0,180.0,60.0,480.0,480.0,60.0,0.0,60.0,Balanced Achiever,Balanced Achiever
3,2024-05-04,210.0,90.0,300.0,45.0,795.0,0.0,0.0,0.0,0.0,Balanced Achiever,Balanced Achiever
4,2024-05-05,150.0,85.0,465.0,0.0,300.0,240.0,200.0,0.0,0.0,Active Explorer,Active Explorer
...,...,...,...,...,...,...,...,...,...,...,...,...
93,2024-08-02,120.0,120.0,75.0,30.0,450.0,90.0,75.0,465.0,45.0,Active Explorer,Active Explorer
94,2024-08-03,120.0,120.0,75.0,30.0,450.0,90.0,75.0,435.0,45.0,Active Explorer,Active Explorer
95,2024-08-04,120.0,120.0,75.0,30.0,420.0,90.0,75.0,435.0,45.0,Active Explorer,Active Explorer
96,2024-08-05,120.0,120.0,75.0,30.0,420.0,90.0,75.0,465.0,45.0,Active Explorer,Active Explorer


In [ ]:
# Compare the predicted and real categories
daily_summary_pivot['Comparison'] = daily_summary_pivot['Predicted_Day_Condition'] == daily_summary_pivot['Day_Condition']
# Summary of comparison
correct_predictions = daily_summary_pivot['Comparison'].sum()
total_predictions = len(daily_summary_pivot)
accuracy = correct_predictions / total_predictions

print(f"Correct Predictions: {correct_predictions}/{total_predictions}")
print(f"Accuracy: {accuracy:.2%}")

Correct Predictions: 96/98
Accuracy: 97.96%


In [ ]:
# Load the TensorFlow Lite model
interpreter = tf.lite.Interpreter(model_path="model.tflite")
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Manually input data for inference
# Replace this with your own input data
input_data = np.array([[168.0,68.0,	1.0,	77.0,	575.0,	188.0,	55.0,	266.0,	42.0]], dtype=np.float32)

# Set input tensor
interpreter.set_tensor(input_details[0]['index'], input_data)

# Run inference
interpreter.invoke()

# Get output tensor
output_data = interpreter.get_tensor(output_details[0]['index'])

# Post-process output data to obtain class labels
class_labels = np.argmax(output_data, axis=1)
# Define class labels
class_labels = ['Active Explorer', 'Balance Achiever', 'Fitness Fanatic', 'Stressful Overload']

# Post-process output data to obtain string class labels
class_indices = np.argmax(output_data, axis=1)
string_labels = [class_labels[i] for i in class_indices]

# Output string class labels
string_labels

['Balance Achiever']